In [1]:
import s3fs
import warnings
import time
import re
import pandas as pd
import numpy as np
import os
import itertools
import multiprocessing
import pickle
import tensorflow as tf
import matplotlib
import matplotlib.pylab as plt
from sklearn import tree
from tensorflow import keras
from functools import partial
from itertools import repeat
from datetime import datetime
from sagemaker import get_execution_role
from dataHandling import dataHandling
from batchTools import batchTools
from gruDataReshaper import gruDataReshaper

In [2]:
# PARAMETERS
DIRHOME = "s3://sagemaker-begas-upload/marketCaster/ver2.4.3"
DAILY_DATE = "2019-10-31"
BATCH_DATE = "2020-01-07"

In [3]:
def correctDate(DF, nAhead):
    tmpDATEDF = DF.copy()
    tmpDATEDF['train_end']=tmpDATEDF['train_end'].apply(lambda x: pd.date_range(x, periods=2, freq='-{}B'.format(nAhead)).strftime('%Y-%m-%d')[1])
    tmpDATEDF['train_start']=tmpDATEDF['train_start'].apply(lambda x: pd.date_range(x, periods=2, freq='-{}B'.format(nAhead)).strftime('%Y-%m-%d')[1])
    tmpDATEDF['valid_end']=tmpDATEDF['valid_end'].apply(lambda x: pd.date_range(x, periods=2, freq='-{}B'.format(nAhead)).strftime('%Y-%m-%d')[1])
    tmpDATEDF['valid_start']=tmpDATEDF['valid_start'].apply(lambda x: pd.date_range(x, periods=2, freq='-{}B'.format(nAhead)).strftime('%Y-%m-%d')[1])
    return tmpDATEDF

In [4]:
def customLoss(y_true, y_pred):
    mean_true = tf.math.reduce_mean(y_true)
    mean_pred = tf.math.reduce_mean(y_pred)
    
    std_true = tf.math.reduce_std(y_true)
    std_pred = tf.math.reduce_std(y_pred)
    
    up = tf.reduce_mean(tf.math.multiply((y_true-mean_true), (y_pred-mean_pred)))
    down = std_true * std_pred
    corr = 1.0-tf.compat.v1.where(tf.math.is_nan(up/down),0.0,up/down)
    
    mserr = tf.math.sqrt(tf.math.reduce_mean(tf.math.square(y_true-y_pred)))
    signerr = 1-tf.math.reduce_mean(tf.math.multiply(tf.math.sign(y_true), tf.math.sign(y_pred)))
    
    return mserr + corr/2 + signerr/2

In [5]:
def modelgen(nTimeSteps, nFeature):
    layer_input = keras.Input(shape=(nTimeSteps, nFeature), name='input')
    layer_state = keras.layers.CuDNNGRU(units=100, return_state=True, name='gru_cell_0')(layer_input)[1]
    layer_gru = keras.layers.CuDNNGRU(units=100, return_sequences=True, name='gru_cell_1')(layer_input, initial_state=layer_state)
    layer_gru = keras.layers.CuDNNGRU(units=100, return_sequences=True, name='gru_cell_2')(layer_gru)
    layer_output = keras.layers.TimeDistributed(
        keras.layers.Dense(units=1, activation='linear',
                          kernel_regularizer=keras.regularizers.l2(l=0.01),
                          kernel_initializer=tf.keras.initializers.Orthogonal(seed=925)), name='output')(layer_gru) 
    gru_model = keras.Model(layer_input, layer_output)
    gru_model.compile(loss=customLoss, optimizer=keras.optimizers.Adam())
    return gru_model

In [6]:
detailDir = '/out/performance_ver2.0/2020-03-03/{}/{:03}/final'
GPUDEVICE = 2

In [7]:
DATEDF = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'out/dateDF.csv'))
DATEDF.sort_values(by='test_end', inplace=True)
DATEDF.reset_index(drop=True, inplace=True)
AHEAD=65
TOTFEATURE = 50
SELFEATURE = 45
TIMESTEPS = 10
TIMEUNIT = 10
NTREE = 10
DATEDF = correctDate(DATEDF, AHEAD)

In [8]:
def dowork(stage):
    VALID_START = DATEDF.iloc[stage,0]
    VALID_END = DATEDF.iloc[stage,1]
    TRAIN_START = DATEDF.iloc[stage,2]
    TRAIN_END = DATEDF.iloc[stage,3]
    TEST_START = DATEDF.iloc[stage,4]
    TEST_END = DATEDF.iloc[stage,5]

    # Read Data
    dataHandler = dataHandling(DIRHOME, MARKET, AHEAD)
    dataHandler.readConvertedData(TEST_END)
    dataHandler.readVarImpData(1000, TEST_END)
    dataHandler.readRawData(DAILY_DATE)
    dataHandler.readEDAData(DAILY_DATE)
    dataHandler.slicingData()
    dataHandler.naRemover()

    # Variable Selection
    varImpDF = dataHandler.varImpDF.copy()
    varImpDF = varImpDF[['var_name', 'correlation','impurity', 'SSE', 'matching_score', 'non_zero_number', 'rank_total']]
    varImpDF['score_1'] = (abs(varImpDF[['correlation']])-abs(varImpDF[['correlation']]).mean())/abs(varImpDF[['correlation']]).std()
    varImpDF['score_2'] = (varImpDF[['impurity']]-varImpDF[['impurity']].mean())/varImpDF[['impurity']].std()
    varImpDF['score_3'] = -(varImpDF[['SSE']]-varImpDF[['SSE']].mean())/(varImpDF[['SSE']].std())
    varImpDF['score_4'] = -(varImpDF[['matching_score']]-varImpDF[['matching_score']].mean())/varImpDF[['matching_score']].std()
    varImpDF['score_5'] = (varImpDF[['non_zero_number']]-varImpDF[['non_zero_number']].mean())/varImpDF[['non_zero_number']].std()
    varImpDF['score'] = varImpDF[['score_1','score_2','score_3','score_4','score_5']].mean(1)
    varImpDF.sort_values('score', ascending=False, inplace=True)
    xVarsNew = varImpDF['var_name'][:TOTFEATURE].tolist()

    # To Numpy
    grudatareshaper = gruDataReshaper(dataHandler.convertedDF, 
                              AHEAD, 
                              dataHandler.yVar, 
                              xVarsNew, 
                              pd.date_range(TRAIN_START, periods=2, freq='-{}B'.format(TIMESTEPS*TIMEUNIT)).strftime('%Y-%m-%d')[1], 
                              TRAIN_END)
    grudatareshaper.minmaxNormalization()
    grudatareshaper.normalizedDF = grudatareshaper.normalizedDF.loc[grudatareshaper.normalizedDF.date_frct<=TEST_END]
    grudatareshaper.normalizedDF.reset_index(drop=True, inplace=True)
    dateDF, xNP, yNP = grudatareshaper.toNumpy(TIMEUNIT, TIMESTEPS, bNormal=True, bYOn=True)
    print(xNP.shape)
    # Seperate Data
    trainXNP = xNP[(dateDF['date_frct']>=TRAIN_START)&(dateDF['date_frct']<=TRAIN_END),:,:]
    trainYNP = yNP[(dateDF['date_frct']>=TRAIN_START)&(dateDF['date_frct']<=TRAIN_END),:,:]
    validXNP = xNP[(dateDF['date_frct']>=VALID_START)&(dateDF['date_frct']<=VALID_END),:,:]
    validYNP = yNP[(dateDF['date_frct']>=VALID_START)&(dateDF['date_frct']<=VALID_END),:,:]
    testXNP = xNP[(dateDF['date_frct']>=TEST_START)&(dateDF['date_frct']<=TEST_END),:,:]
    testYNP = yNP[(dateDF['date_frct']>=TEST_START)&(dateDF['date_frct']<=TEST_END),:,:]
    flagDateDF = dateDF.loc[(dateDF['date_frct']>=TEST_START)&(dateDF['date_frct']<=TEST_END)]
    dateDF = dateDF.loc[(dateDF['date_frct']>=VALID_START)]

    # Random Forest (Variable Sampling)
    colNumLT = [np.random.choice(TOTFEATURE, SELFEATURE, replace=False) for x in range(NTREE)]

    with tf.device('/gpu:{}'.format(GPUDEVICE)):
        # gpu setting
        config = tf.ConfigProto()
        config.gpu_options.allow_growth=True
        sess = tf.Session(config=config)  
        tf.keras.backend.set_session(sess)
        tf.set_random_seed(925)
        # Model + Training
        modelLT = [modelgen(TIMESTEPS, SELFEATURE) for x in range(NTREE)]
        earlyStop = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=30)
        tf.set_random_seed(925)
        for x in range(NTREE):
            print('[ Model {:02} ]'.format(x))
            modelLT[x].fit(trainXNP[:,:,colNumLT[x]], trainYNP,
                           validation_data = (validXNP[:,:,colNumLT[x]], validYNP),
                           batch_size=20,
                           epochs=200,
                           callbacks=[earlyStop],
                           verbose=0)
            print('  - complete!\n')    

        # Compute Training Error
        err = [((modelLT[x].predict(trainXNP[:,:,colNumLT[x]])[:,-1,-1]-trainYNP[:,-1,-1])**2).sum() for x in range(10)]
        weight = (1-np.array(err))/(1-np.array(err)).sum()

        for x in range(10):
            inputs = np.where(testXNP[:,:,colNumLT[x]]>1,1,testXNP[:,:,colNumLT[x]])
            inputs = np.where(inputs<0, 0, inputs)
            if x==0:
                prediction0 = modelLT[x].predict(inputs)[:,-1,-1] * weight[x]
                prediction1 = modelLT[x].predict(testXNP[:,:,colNumLT[x]])[:,-1,-1] * weight[x]
            else:
                prediction0 += modelLT[x].predict(inputs)[:,-1,-1] * weight[x]
                prediction1 += modelLT[x].predict(testXNP[:,:,colNumLT[x]])[:,-1,-1] * weight[x]

        prediction = prediction0*0.8 + prediction1*0.2
        #tf.keras.backend.clear_session()

    ## Save
    predDF = flagDateDF.copy()
    predDF['prediction'] = prediction
    tmpY = re.sub('_last_[0-9]{3}$|_last_[0-9]{2}$|_last_[0-9]{1}$','_last', dataHandler.yVar[0])
    predDF = predDF.merge(dataHandler.rawDF.rename(columns={'date':'date_base',
                                                           tmpY:'y_base'}), 'left')
    tmp = dataHandler.rawDF.rename(columns={'date':'date_frct',tmpY:'y_frct'})
    tmp['moving_std'] = tmp['y_frct'].rolling(window=int(np.where(AHEAD==1, 5, AHEAD))).std()
    tmp['lower_bound_accuracy'] = tmp['y_frct'] - 1.644854*tmp['moving_std']
    tmp['upper_bound_accuracy'] = tmp['y_frct'] + 1.644854*tmp['moving_std']
    tmp = tmp[['date_frct', 'y_frct', 'lower_bound_accuracy', 'upper_bound_accuracy']]
    predDF = predDF.merge(tmp, 'left')

    batchtools = batchTools('2020-01-16')
    predDF = batchtools.reconvertData(tmpY, predDF, dataHandler.edaDF)
    ## Save - Path
    saveDir = os.path.dirname(os.getcwd()) + detailDir.format(MARKET,AHEAD) 
    if not os.path.isdir(saveDir):
        os.makedirs(saveDir)
    predDF.sort_values(by=['date_frct'], inplace=True)
    predDF.to_csv(saveDir+'/stage_{:03}.csv'.format(stage),index=False)
    print('    - Complete!\n ')

In [9]:
for MARKET in ['kospi_index','spx_index','shcomp_index',
               'gtkrw10y_corp','gtusd10y_govt','gtcny10y_corp',
               'dxy_curncy','cl1_comdty','xau_curncy']:
    print(MARKET)
    pool = multiprocessing.Pool(processes=8)
    pool.map(dowork, np.arange(15))
    pool.close()
    pool.join()
    end_time = time.time()

kospi_index
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:21) [ Read Converted Data ]
(2020-03-04 12:05:25) [ Read Variables Importance Data ]
(2020-03-04 12:05:25) [ Read Raw Data ]
(2020-03-04 12:05:25) [ Read EDA Data ]
(2020-03-04 12:05:25) [ Remove Non-Available Data ]
                        * 65 rows are removed!

(2020-03-04 12:05:26) [ Read Variables Importance Data ]
(2020-03-04 12:05:26) [ Read Variables Importance Data ]
(2020-03-04 12:05:26) [ Read Raw Data ]
(2020-03-04 12:05:26) [ Read Variables Importance Data ]
(2020-03-04 12:05:26) [ Read EDA Data ]
(2020-03-04 12:05:26) [ Read Variables Importance Data ]
(2020-03-04 12:05:26) [ Read Raw Data ]
(2020-03-04 12:05:26) [ Read Raw Da

  - complete!

[ Model 06 ]
  - complete!

[ Model 01 ]
  - complete!

[ Model 01 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 02 ]
  - complete!

  - complete!

[ Model 05 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 04 ]
    - Complete!
 
  - complete!

[ Model 05 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 05 ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 08 ]
  - complete!

  - complete!

[ Model 05 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 07 ]


  - complete!

[ Model 01 ]
  - complete!

[ Model 05 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 01 ]
  - complete!

[ Model 02 ]
  - complete!

  - complete!

[ Model 05 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 06 ]
    - Complete!
 
(2020-03-04 12:33:03) [ Read Converted Data ]
  - complete!

[ Model 02 ]
(2020-03-04 12:33:08) [ Read Variables Importance Data ]
(2020-03-04 12:33:09) [ Read Raw Data ]
(2020-03-04 12:33:09) [ Read EDA Data ]
(2020-03-04 12:33:09) [ Remove Non-Available Data ]
                        * 65 rows are removed!

    - Reshape Data (converting data frame to numpy.)
  - complete!

[ Model 03 ]
(4824, 10, 50)
  - complete!

[ Model 06 ]
[ Model 00 ]
  - complete!

    - Complete!
 
(2020-03-04 12:33:47) [ Read Converted Data ]
(2020-03-04 12:33:52) [ Read Variables Importance Data ]
(2020-03-04 12:33:52) [ Read Raw Data ]
(2020-03-04 12:33:52) [ Read EDA Data ]
(2020-03-04 12:33:52) [ Remove Non-Available Data ]
                        * 6

  - complete!

[ Model 09 ]
(2020-03-04 12:50:41) [ Read Variables Importance Data ]
(2020-03-04 12:50:41) [ Read Raw Data ]
(2020-03-04 12:50:41) [ Read EDA Data ]
(2020-03-04 12:50:41) [ Remove Non-Available Data ]
                        * 65 rows are removed!

  - complete!

  - complete!

[ Model 05 ]
    - Reshape Data (converting data frame to numpy.)
  - complete!

[ Model 07 ]
(4629, 10, 50)
  - complete!

[ Model 03 ]
[ Model 00 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 07 ]
  - complete!

  - complete!

[ Model 06 ]
  - complete!

[ Model 04 ]
    - Complete!
 
(2020-03-04 12:51:33) [ Read Converted Data ]
(2020-03-04 12:51:38) [ Read Variables Importance Data ]
(2020-03-04 12:51:38) [ Read Raw Data ]
(2020-03-04 12:51:38) [ Read EDA Data ]
(2020-03-04 12:51:38) [ Remove Non-Available Data ]
                        * 65 rows are removed!

  - complete!

[ Model 08 ]
  - complete!

    - Reshape Data (converting data frame to numpy.)
  - complete!

[ Model 01 ]
(46


[ Model 03 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 05 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 05 ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 08 ]
  - complete!

  - complete!

[ Model 05 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 07 ]
    - Complete!
 
(2020-03-04 13:10:44) [ Read Converted Data ]
(2020-03-04 13:10:49) [ Read Variables Importance Data ]
(2020-03-04 13:10:49) [ Read Raw Data ]
(2020-03-04 13:10:49) [ Read EDA Data ]
(2020-03-04 13:10:49) [ Remove Non-Available Data ]
                  

...

(2020-03-04 13:41:07) [ Read EDA Data ]
(2020-03-04 13:41:07) [ Remove Non-Available Data ]
                        * 1530 rows are removed!

(2020-03-04 13:41:08) [ Read Variables Importance Data ]
(2020-03-04 13:41:08) [ Read Variables Importance Data ]
(2020-03-04 13:41:08) [ Read Variables Importance Data ]
(2020-03-04 13:41:08) [ Read Variables Importance Data ]
(2020-03-04 13:41:08) [ Read Raw Data ]
(2020-03-04 13:41:08) [ Read Raw Data ]
(2020-03-04 13:41:08) [ Read EDA Data ]
(2020-03-04 13:41:08) [ Read Raw Data ]
(2020-03-04 13:41:08) [ Read EDA Data ]
(2020-03-04 13:41:08) [ Read Raw Data ]
(2020-03-04 13:41:08) [ Remove Non-Available Data ]
(2020-03-04 13:41:08) [ Read EDA Data ]
(2020-03-04 13:41:08) [ Read EDA Data ]
(2020-03-04 13:41:08) [ Remove Non-Available Data ]
(2020-03-04 13:41:08) [ Remove Non-Available Data ]
(2020-03-04 13:41:08) [ Remove Non-Available Data ]
                        * 1530 rows are removed!

(2020-03-04 13:41:08) [ Read Variables Importance Da

  - complete!

  - complete!

[ Model 08 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 01 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 07 ]
  - complete!

  - complete!

[ Model 09 ]
  - complete!

[ Model 05 ]
    - Complete!
 
  - complete!

[ Model 03 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 02 ]
  - complete!

  - complete!

[ Model 07 ]
  - complete!

[ Model 09 ]
    - Complete!
 
  - complete!

[ Model 09 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 04 ]
  - complete!

[ Model 08 ]
  - complete!

  - complete!

  - complete!

[ Model 05 ]
    - Complete!
 
  - complete!

[ Model 09 ]
  - complete!

[ Model 04 ]
  - complete!

  - complete!

[ Model 06 ]
    - Complete!
 
  - complete!

[ Model 07 ]
  - complete!

[ Model 05 ]
    - Complete!
 
  - complete!

[ Model 08 ]
    - Complete!
 
  - complete!

[ Model 06 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 07 ]
  - complete!

  -

(2020-03-04 14:06:06) [ Remove Non-Available Data ]
                        * 65 rows are removed!

    - Reshape Data (converting data frame to numpy.)
  - complete!

[ Model 01 ]
    - Reshape Data (converting data frame to numpy.)
(4759, 10, 50)
[ Model 00 ]
(4824, 10, 50)
(4889, 10, 50)
  - complete!

[ Model 02 ]
[ Model 00 ]
[ Model 00 ]
  - complete!

[ Model 02 ]
    - Complete!
 
(2020-03-04 14:06:33) [ Read Converted Data ]
(2020-03-04 14:06:38) [ Read Variables Importance Data ]
(2020-03-04 14:06:38) [ Read Raw Data ]
(2020-03-04 14:06:38) [ Read EDA Data ]
(2020-03-04 14:06:38) [ Remove Non-Available Data ]
                        * 65 rows are removed!

    - Reshape Data (converting data frame to numpy.)
  - complete!

[ Model 01 ]
(4954, 10, 50)
    - Complete!
 
  - complete!

[ Model 03 ]
  - complete!

[ Model 01 ]
[ Model 00 ]
  - complete!

[ Model 01 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 02 ]
  - complete!

[ Model 01 ]
  

(4629, 10, 50)
  - complete!

[ Model 07 ]
[ Model 00 ]
(2020-03-04 14:21:52) [ Read Variables Importance Data ]
(2020-03-04 14:21:52) [ Read Raw Data ]
(2020-03-04 14:21:52) [ Read EDA Data ]
(2020-03-04 14:21:52) [ Remove Non-Available Data ]
                        * 65 rows are removed!

    - Reshape Data (converting data frame to numpy.)
  - complete!

[ Model 07 ]
  - complete!

[ Model 02 ]
  - complete!

(4694, 10, 50)
[ Model 00 ]
    - Complete!
 
(2020-03-04 14:22:18) [ Read Converted Data ]
(2020-03-04 14:22:23) [ Read Variables Importance Data ]
(2020-03-04 14:22:24) [ Read Raw Data ]
(2020-03-04 14:22:24) [ Read EDA Data ]
(2020-03-04 14:22:24) [ Remove Non-Available Data ]
                        * 65 rows are removed!

  - complete!

[ Model 07 ]
    - Reshape Data (converting data frame to numpy.)
  - complete!

[ Model 08 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 01 ]
(4759, 10, 50)
[ Model 00 ]
  - complete!

[ Model 04 ]
    - Complete!
 
(2020-03-04 14:


[ Model 05 ]
  - complete!

[ Model 03 ]
  - complete!

[ Model 05 ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 05 ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 08 ]
  - complete!

  - complete!

[ Model 04 ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 06 ]
  - complete!

  - complete!

[ Model 07 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 09 ]
  - complete!

[ Model 06 ]
  - complete!

[ Model 07 ]
  - complete!

[ Model 05 ]
  - complete!

    - Complete!
 
(2020-03-04 14:41:36) [ Read Converted Data ]
  - complete!

[ Model 08 ]
  - complete!

[ Model 07 ]
(2020-03-04 14:41:42) [ Read Variables Importance Data ]
(2020-03-04 14:41:42) [ Read Raw Data ]
(2020-03-04 14:41:42) [ Read EDA Data ]
(2020-03-04 14:41:42) [ Remove Non-Available Data ]
                        * 65 rows are remove